https://learn.microsoft.com/en-us/azure/ai-services/openai/gpt-v-quickstart?tabs=image%2Ccommand-line&pivots=programming-language-python

https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/gpt-with-vision?tabs=python%2Csystem-assigned%2Cresource#use-a-local-image


USE AZURE-OPENAI ENV python 3.11.8


pip install gradio==3.48.0


pip install pydantic==2.6.1

pip install openai==1.59.6

https://github.com/openai/openai-python

https://github.com/balakreshnan/Samples2023/blob/main/AzureML/gpt4turbopdf.md

In [30]:
import base64
import datetime
import glob
import gradio as gr
import json
import openai
import os
import requests
import sys

import PyPDF2

from dotenv import load_dotenv
from io import BytesIO
from PIL import Image
from IPython.display import display

! pip install "openai>=1.0.0,<2.0.0"

! pip install python-dotenv

! pip install "azure-identity>=1.15.0"



https://cookbook.openai.com/examples/azure/functions

In [31]:
import os
import openai
import dotenv

dotenv.load_dotenv()

use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

if not use_azure_active_directory:
    endpoint = "https://llmgpt.openai.azure.com/"
    api_key = "f428a7b27d9c49c09ddc2319b7fb87e8"
    # set the deployment name for the model we want to use
    #deployment = "GPT4Vision"
    #deployment = "GPT4-32Vision"
    #deployment = "CODE0613"
    deployment ="4OMINI"
    #####deployment ="EMBEDDING03"

    client = openai.AzureOpenAI(
        #base_url=f"{endpoint}/openai/deployments/{deployment}/extensions",
        base_url=f"{endpoint}/openai/deployments/{deployment}/",
        api_key=api_key,
        api_version="2023-09-01-preview" # this might change in the future
        ### ===> future? api_version = '2024-02-15-preview' ####
    )

In [32]:
#pdf_file = "2409.00082v1.pdf"
#pdf_file = "2309.17421v2.pdf"
#pdf_file = "MEDREC.pdf"
#pdf_file = "DossierLG2.pdf"
# pdf_file = "beroepsprofielen.pdf"
pdf_file = "MEDLO_All1.pdf"

In [33]:
processed_text_list = []
# Open the PDF file in binary mode
with open(pdf_file, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    #print(pdf_reader)
    # Iterate through each page and extract text
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        page_text = page.extract_text()
        processed_text_list.append(page_text)

# Combine all AI-processed text into a single string
combined_text = "\n".join(processed_text_list)

In [34]:
display(combined_text)

"BewonernummerActiviteitLocatieBetrokkenheidMate van sociale interactieInteractie met wieSoort sociale interactieFysieke inspanningStemming631. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer1731. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer2131. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer2331. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer431. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer131. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer716. Huishoudelijke activiteiten (alles)1. in een gezamenlijke ruimte op de woonvorm1. actief (duidelijke participatie met de activiteit)3. tweeweg interactie4. met anderen5. positief sociaal5. Lopen (licht tuinieren)5. kleine signalen van gelukkig zijn1531. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer831. Rusten of bewust slapen (alleen activiteit en 

In [35]:
import pandas as pd

# Create a DataFrame with a single column containing the text
df = pd.DataFrame({'Text': [combined_text]})

# Display the DataFrame
print(df)




                                                Text
0  BewonernummerActiviteitLocatieBetrokkenheidMat...


In [36]:
# combined_text ='Radboudumc Locatie\nGeert Grooteplein Zuid 10\n6525 GA NijmegenJoling, F.J.M.\nMDN: 0470330, Geb.dat.: 7-9-2005, Geslacht: V\nBezoekdatum: 23-7-2024\n23-07-2024 - Afspraak op Interne Geneeskunde\nMedische notities\nBovengenoemde patiënt  zagen wij via videoconsult op de reis- en vaccinatiepoli voor  vaccinatieadvies.'

In [37]:
### https://medium.com/@lisakim01/how-to-use-gpt-4o-api-understanding-max-context-length-06a754a882bc#:~:text=The%20documentation%20and%20various%20community,the%20API%20is%208192%20tokens.

In [38]:
# Define the document content
document_content = combined_text

# Split the document into chunks of 4096 tokens (half of 8192 for safety)
chunk_size = 4096
chunks = []
current_chunk = ""
token_count = 0

for word in document_content.split():
    if token_count + len(word.split()) > chunk_size:
        chunks.append(current_chunk)
        current_chunk = word
        token_count = len(word.split())
    else:
        current_chunk += " " + word
        token_count += len(word.split())

if current_chunk:
    chunks.append(current_chunk)

In [39]:
display(chunks)

[" BewonernummerActiviteitLocatieBetrokkenheidMate van sociale interactieInteractie met wieSoort sociale interactieFysieke inspanningStemming631. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer1731. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer2131. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer2331. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer431. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer131. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer716. Huishoudelijke activiteiten (alles)1. in een gezamenlijke ruimte op de woonvorm1. actief (duidelijke participatie met de activiteit)3. tweeweg interactie4. met anderen5. positief sociaal5. Lopen (licht tuinieren)5. kleine signalen van gelukkig zijn1531. Rusten of bewust slapen (alleen activiteit en locatie)2. in de eigen kamer831. Rusten of bewust slapen (alleen activiteit e

In [40]:
for chunk in chunks:
    messages = [
    {
        "role": "system",
        "content": """
        You are a Assistant, a backend processor.
        - User input DUTCH raw text extracted from a PDF page by PyPDF2.
        - Answer in DUTCH.
        """
    },
    {
        "role": "user",
        "content": "Summarize the content in 200 words:" + chunk
    }
    
    ]



In [41]:
messages = [
    {
        "role": "system",
        "content": """
        You are a Assistant, a backend processor.
        - User input DUTCH raw text extracted from a PDF page by PyPDF2.
        - Answer in DUTCH.
        """
    },
    {
        "role": "user",
        "content": "Summarize the content in 200 words:" + combined_text
    }
    
]

In [42]:
response = client.chat.completions.create(
    model= deployment, # model = "deployment_name".
    messages=messages,
    max_tokens = 4096
)

print(response.choices[0].message.content)

De tekst geeft een uitgebreide beschrijving van de activiteiten, interactie en stemming van bewoners in een woonvorm. De bewoners nemen gedurende hun dagen deel aan diverse activiteiten zoals rusten, huishouden, maaltijden, verzorgingsactiviteiten, en sociale interacties. Er zijn veel momenten van alleen zijn in eigen kamers, met een nadruk op rusten of slapen. De interacties zijn een belangrijk aspect; deze variëren van alleen met personeel, interactie met andere bewoners, tot gesprekken en gezamenlijke activiteiten.

De mate van sociale interactie varieert van afwezig tot actief, waarbij bijvoorbeeld het deelnemen aan maaltijden en huishoudelijke taken vaak positieve sociale interacties oplevert, ook al is er soms sprake van een eenzijdige interactie. De stemming van bewoners wordt gemonitord met signalen van geluk of neutrale reacties. Over het algemeen lijkt de betrokkenheid van bewoners tijdens activiteiten te fluctueren, met een tendens naar zowel actieve deelname in sociale zin 

In [28]:
print(response.usage)

CompletionUsage(completion_tokens=243, prompt_tokens=106650, total_tokens=106893, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


In [29]:
##### Expected a string with maximum length 1048576
##### 
##### "This model's maximum context length is 128000 tokens. However, your messages resulted in 272884 tokens. Please reduce the length of the messages."